In [ ]:
# to print all output for a cell instead of only last one 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
string = ' ciao mi ✅ riccardo :) '

print(string)

In [ ]:
import pandas as pd 
tweets_df = None 
pd.Series(' '.join(tweets_df[tweets_df['label']=='1']['tweet']).lower().split()).value_counts()[:1000].to_dict().keys()

In [ ]:
import reticker 
# reticker.config.BLACKLIST.add("RT")
config = reticker.TickerMatchConfig(unprefixed_uppercase=False,prefixed_titlecase=False)
extractor = reticker.TickerExtractor(match_config=config)

df1 = tweets_df[tweets_df['tweet'].str.contains('(?<!\S)\$[a-zA-Z]+(?!\S)', regex=True)]
df3 = tweets_df[tweets_df['tweet'].str.contains('[$][A-Za-z][\S]', regex=True)]
df4 = tweets_df[tweets_df['tweet'].str.contains(extractor.pattern, regex=True)]



In [ ]:
l = tweets_df['tweet'].str.findall("(?<!\S)\$[0-9]+(?!\S)")

lb = [el for el in l if el]

l2 = tweets_df['tweet'].str.findall("\\B\$[0-9]+(?:\.\d+)?[k+]?")

lb2 = [el for el in l2 if el]

import itertools
lb = list(itertools.chain(*lb))
lb2 = list(itertools.chain(*lb2))

print(list(set(lb2) - set(lb)))

In [ ]:
EMOTICONS = [':\)','\(:',':-\)','\s:\^\)\s','\s=\]\s','\s:-\(\s','\s:\(\s','\s:\(\s','\^_\^','\s:\'\(\s','\s:-D\s']

tweets_df2 = tweets_df.copy(deep=True)

In [ ]:
tk.tokenize("what's going on over there, they're my friends")

In [ ]:
URL_KEYWORDS = ['https', 'http', '.com']
sentence = 'ciao'
sentence = [' url ' if any(k in word for k in URL_KEYWORDS) else word for word in sentence]


In [ ]:
import re 
def replace(word : str):
    if not word.isascii():
        return ['']
    elif bool(re.search(r'http[s]?|.com',word)):
        return ['url']
    elif bool(re.search(r'\d',word)):
        return ['number']
    elif bool(re.search(r'haha|ahah|jaja|ajaj',word)):
        return ['ahah']
    elif bool(re.search(r'\n',word)):
        return ['']
    elif bool(re.search('-',word)):
        return re.sub('-',' ',word).split()
    else :
        return re.split("(')",word) 
        # return [word]

a = "ciao mi chiamo mario batti5 ahahhah , guarda il mio-profilo https:// 𝙒𝙞𝙨𝙝𝙞𝙣𝙜, su-per-be-llo. There's that"

print([w for word in a.split() for w in replace(word) ])



In [ ]:
import torch 

a = torch.randint(0,9,(2,5,10))
a
a.shape

b = torch.concat((a[-2,:,:],a[-1,:,:]),dim=1)
b
b.shape


In [ ]:
import gdown 
emb_model_cached_path = ''
id = "1DprdHGocFXJ9swnb2pDJJxHw5QR810LS"
gdown.download(id=id,output=str(emb_model_cached_path))


# emb_model = gloader.load(emb_model_download_path)
# print('saving embeddings to file')  
# emb_model.save_word2vec_format(emb_model_cached_path, binary=True)   

In [ ]:
import numpy as np 
import torch.nn as nn 

class Custom_model(nn.Module):
    """
        Class defining our model architecture  
    """
    def __init__(self, emb_matrix: np.ndarray, cfg : dict, device) :
        super().__init__()

        self.cfg = cfg
        self.device = device 
        
        self.embedding_layer, self.word_embedding_dim = self.build_emb_layer(emb_matrix,cfg['pad_idx'], cfg['freeze_embedding'])

        self.rnn = nn.LSTM(self.word_embedding_dim, cfg['hidden_dim'], batch_first = True) 
            
        self.drop_layer = nn.Dropout(cfg['dropout_p']) 

        self.classifier = nn.Linear(self.word_embedding_dim,1)   

    def name(self):
        return 'GiacomoSingleTweet_model'

    def build_emb_layer(self, weights_matrix: np.ndarray, pad_idx : int, freeze : bool):
    
        matrix = torch.from_numpy(weights_matrix).to(self.device)   #the embedding matrix 
        _ , embedding_dim = matrix.shape 

        emb_layer = nn.Embedding.from_pretrained(matrix, freeze=freeze, padding_idx = pad_idx)   #load pretrained weights in the layer and make it non-trainable (TODO: trainable ? )
        
        return emb_layer, embedding_dim
        

    def words_embedding(self, word_idxs):   #get embedding vectors for each token in sentence 
        """
            Input:  [batch_size, num_tokens]
            Output: [batch_size, num_tokens, embedding_dim]
        """
        return self.embedding_layer(word_idxs)
    
    def sentence_embedding(self, embeddings, sentence_lenghts):     #compute sentence embedding 
        """
            Input:  [batch_size, num_tokens, embedding_dim]
            Output: [batch_size, embedding_dim]
        """
        #take as sentence embedding the average of all the states of the rnn corresponing to each word 

        packed_embeddings = pack_padded_sequence(embeddings, sentence_lenghts, batch_first=True, enforce_sorted=False)
        packed_out, _  = self.rnn(packed_embeddings)

        unpacked_out, l = pad_packed_sequence(packed_out,batch_first=True)

        avg = unpacked_out.sum(dim=1).div(sentence_lenghts.unsqueeze(dim=1).to(self.device))

        return avg

    def forward(self, batch_data):

        tweets = batch_data['tweets']           # [batch_size, num_tokens]
        tweet_lengths = batch_data['lengths']   # [batch_size]
       
        #embed each word in a sentence with a 300d vector 
        word_emb_tweet = self.words_embedding(tweets)

        #compute sentence embedding
        sentence_emb_tweet = self.sentence_embedding(word_emb_tweet,tweet_lengths)

        #eventual dropout 
        if self.cfg['dropout']: 
            sentence_emb_tweet = self.drop_layer(sentence_emb_tweet)

        #final classification 
        predictions = self.classifier(sentence_emb_tweet)

        predictions = predictions

        return predictions

In [ ]:
import json 
from pathlib import Path
import pandas as pd 

# PATHS 
BASE_PATH = Path(*Path().absolute().parts[:-1])
DATA_FOLDER = BASE_PATH / 'data' # directory containing the notebook

json_file_path_train = DATA_FOLDER / 'Twibot-20/train.json'
json_file_path_val = DATA_FOLDER / 'Twibot-20/dev.json'
json_file_path_test = DATA_FOLDER / 'Twibot-20/test.json'

with open(json_file_path_train, 'r') as tr:
     contents = json.loads(tr.read())
     train_df = pd.json_normalize(contents)
     train_df['split'] = 'train'

with open(json_file_path_val, 'r') as vl:
     contents = json.loads(vl.read())
     val_df = pd.json_normalize(contents) 
     val_df['split'] = 'val'

with open(json_file_path_test, 'r') as ts:
     contents = json.loads(ts.read())
     test_df = pd.json_normalize(contents) 
     test_df['split'] = 'test'

df = pd.concat([train_df,val_df,test_df],ignore_index=True) # merge three datasets
df.dropna(subset=['tweet'], inplace=True)  # remove rows withot any tweet 
df.set_index(keys='ID',inplace=True) # reset index

# split dataframe in two : tweet and account data 
tweets_df = df[['tweet','label','split']].reset_index()
tweets_df = tweets_df.explode('tweet').reset_index(drop=True)
tweets_df.rename(columns={"ID": "account_id"}, inplace=True)

account_df = df.drop('tweet',axis=1).reset_index()
account_df.rename(columns={"ID": "account_id"}, inplace=True)


In [ ]:
NEWLINE = r"\n"
RETWEET = r"^RT (?:@[\w_]+):"
CASHTAG = r"(?<!\S)\$[A-Z]+(?:\.[A-Z]+)?(?!\S)"
EMAIL = r"""[\w.+-]+@[\w-]+\.(?:[\w-]\.?)+[\w-]"""
MONEY = r"[$£][0-9]+(?:[.,]\d+)?[Kk+BM]?|[0-9]+(?:[.,]\d+)?[Kk+BM]?[$£]"
NUMBER = r"""(?<!\S)(?:[+\-]?\d+(?:%|(?:[,/.:-]\d+[+\-]?)?))"""
HASHTAG = r"""(?:\#+[\w_]+[\w\'_\-]*[\w_]+)"""
HANDLE = r"""(?:@[\w_]+)"""


tweets_df['tweet'] = tweets_df['tweet'].replace(NUMBER,' NUMBER ',regex=True,inplace=False)

In [ ]:
tweets_df[tweets_df['account_id']=='1023986311433048064']['tweet'].tolist()[111]